# LeNet-5

**Korzystając z notebooka o sieciach konwolucyjnych oraz notebooka o płytkiej sieci w Tensorflow, za pomocą biblioteki Tensorflow zaprojektuj konwolucyjną sieć neuronową o architekturze zbliżonej do słynnej sieci [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) której sukcesy w rozpoznawaniu obrazów rozpoczęły na przełomie lat 80. i 90. boom na głębokie uczenie, i użyj jej do predykcji zbioru FashionMnist, zgodnie z następującym opisem:**

|Nr warstwy|Rodzaj|Rozmiar wyjścia|Liczba filtrów|Rozmiar jądra (`kernel_size`)|Krok (`stride`)|Padding|Funkcja aktywacji|
|----------|---|---------------|--------------|-----------------------|---------------|-------|-----------------|
|1|splotowa|$28\times 28$|6|$5\times 5$|1|2|tanh|
|2|avgerage pooling|$14\times 14$|6|$2\times 2$|2|0|sigmoid|
|3|splotowa|$10\times 10$|16|$5\times 5$|1|0|tanh|
|4|average pooling|$5\times 5$|16|$2\times 2$|2|0|sigmoid|
|5|splotowa|$1\times 1$|120|$5\times 5$|1|0|tanh|
|6|pełna|84|||||tanh|
|7|pełna|10|||||softmax|

**Kolejne kroki:**
 - załadowanie zbioru
 - wydzielenie zbioru walidacyjnego
 - normalizacja i kategoryzacja danych (pomiędzy normalizacją i kategoryzacją przekształć zarówno zbiór treningowy jak i walidacyjny w tensory o odpowiednich wymiarach za pomocą funkcji _expand_dims(dataset, axis=3)_)
 - zaprojektowanie sieci (nie zapominaj o spłaszczeniu w odpowiednim momencie)
 - zbudowanie modelu z dowolną funkcją kosztu dla problemów wieloklasowych, dowolnym optymalizatorem i trafnością jako metryką
 - nauczenie sieci w maksymalnie stu epokach z zastosowaniem wczesnego zatrzymania (najlepiej użyć wbudowanego, z dowolną wartością do monitorowania)
 - ewaluacja modelu na zbiorze walidacyjnym
 - wyświetlenie 10 niepoprawnie zaklasyfikowanych próbek w postaci: prawdziwa prognozowana, np. 2 7

In [84]:
import tensorflow
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt
import numpy as np

## 1. Load data

In [85]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

## 2. Normalize and categorize the data

In [86]:
train_images, test_images = train_images / 255.0, test_images / 255.0

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

## 3. Convert labels to categorical

In [87]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## 4. Define the model

In [88]:
model = models.Sequential()
model.add(layers.Conv2D(6, (5, 5), strides=(1, 1), padding='same', activation='tanh', input_shape=(28, 28, 1)))
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, (5, 5), strides=(1, 1), padding='valid', activation='tanh'))
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, (5, 5), strides=(1, 1), padding='valid', activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))

## 5. Compile the model

In [89]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 6. Training, Evaluation, Early stopping

In [90]:
PATIENCE = 5
WAIT = 0
EPOCHS_NUM = 100

for epoch in range(EPOCHS_NUM):
    model.fit(train_images, train_labels, epochs=1)

    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        WAIT = 0
    else:
        WAIT += 1
        if WAIT >= PATIENCE:
            print(f"Early stopping on epoch {epoch}")
            break
    print(f"Epoch {epoch+1}/{EPOCHS_NUM} Test loss: {test_loss} Test accuracy: {test_acc}")

1875/1875 [==============================] - 6s 3ms/step - loss: 1.0360 - accuracy: 0.5998
313/313 - 1s - loss: 0.6844 - accuracy: 0.7435 - 542ms/epoch - 2ms/step
Epoch 1/100 Test loss: 0.6844043731689453 Test accuracy: 0.7434999942779541
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6534 - accuracy: 0.7464
313/313 - 0s - loss: 0.6562 - accuracy: 0.7491 - 458ms/epoch - 1ms/step
Epoch 2/100 Test loss: 0.6561905145645142 Test accuracy: 0.7491000294685364
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5900 - accuracy: 0.7712
313/313 - 0s - loss: 0.5693 - accuracy: 0.7906 - 474ms/epoch - 2ms/step
Epoch 3/100 Test loss: 0.569276750087738 Test accuracy: 0.7906000018119812
 352/1875 [====>.........................] - ETA: 4s - loss: 0.5503 - accuracy: 0.7859

## 7. Prediction

In [ ]:
predictions = model.predict(test_images)
predictions

313/313 [==============================] - 0s 1ms/step


## 8. Ten wrong predictions

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(test_labels, axis=1)
wrong_indices = []
for i in range(len(actual_labels)):
    actual = actual_labels[i]
    pred = predicted_labels[i]
    if actual != pred:
        wrong_indices.append(i)
for i in wrong_indices[:10]:
    print(f"Actual: {actual_labels[i]}, Predicted: {predicted_labels[i]}")

Actual: 9, Predicted: 7
Actual: 6, Predicted: 2
Actual: 4, Predicted: 2
Actual: 6, Predicted: 4
Actual: 4, Predicted: 6
Actual: 5, Predicted: 7
Actual: 9, Predicted: 5
Actual: 4, Predicted: 6
Actual: 6, Predicted: 4
Actual: 0, Predicted: 3
